In [2]:
import sys

from os.path import basename
import import_ipynb
from Sampler import *
from Pix2Code import *


from Utility import *
from Compiler import *

In [3]:
trained_weights_path = "AndroidModel"
trained_model_name = "pix2code"

output_path = "CodeAndroid"
search_method = "greedy"
file_name = "Output"
meta_dataset = np.load("{}/meta_dataset.npy".format(trained_weights_path))
input_shape = meta_dataset[0]
output_size = meta_dataset[1]

ValueError: Object arrays cannot be loaded when allow_pickle=False

In [ ]:


model = pix2code(input_shape, output_size, trained_weights_path)
model.load(trained_model_name)

sampler = Sampler(trained_weights_path, input_shape, output_size, CONTEXT_LENGTH)




In [ ]:
#IMAGE_SIZE = 256
input_path = "0B6C3747-EDE6-46C5-A2A3-0634B8D27876.png"
evaluation_img = Utils.get_preprocessed_img(input_path, IMAGE_SIZE)

if search_method == "greedy":
    result, _ = sampler.predict_greedy(model, np.array([evaluation_img]))
    print("Result greedy: {}".format(result))
else:
    beam_width = int(search_method)
    print("Search with beam width: {}".format(beam_width))
    result, _ = sampler.predict_beam_search(model, np.array([evaluation_img]), beam_width=beam_width)
    print("Result beam: {}".format(result))


with open("{}/{}.gui".format(output_path, file_name), 'w') as out_f:
    out_f.write(result.replace(START_TOKEN, "").replace(END_TOKEN, ""))

print("Converted")

In [ ]:


TEXT_PLACE_HOLDER = "[TEXT]"
ID_PLACE_HOLDER = "[ID]"

dsl_path = "assets/android-dsl-mapping.json"
compiler = Compiler(dsl_path)


def render_content_with_text(key, value):
    value = value.replace(TEXT_PLACE_HOLDER, Utility.get_random_text(length_text=5, space_number=0))
    while value.find(ID_PLACE_HOLDER) != -1:
        value = value.replace(ID_PLACE_HOLDER, Utility.get_android_id(), 1)
    return value


input_file_path = "{}/{}.gui".format(output_path, file_name)
output_file_path = "{}/{}.xml".format(output_path, file_name)

print(input_file_path, output_file_path)
compiler.compile(input_file_path, output_file_path, rendering_function=render_content_with_text)
print("Completed")